In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH15-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-RAG


# MultiVectorretriever

1. 개요 : 문서를 여러 벡터로 저장하고 관리하여 검색 효율성 향상 
2. 문서당 여러 벡터를 생성하는 방법 
    1) 작은 청크 생성
        - 문서를 더 작은 단위로 나눈 후, 각 청크에 대해 별도의 임베딩 생성 
        - 세부 정보에 대한 탐색 용이
        - `ParentDocumentRetriever` 통해 구현  
    2) 요약 임베딩 
        - 각 문서의 요약을 생성하고, 이 요약을 임베딩 
        - 요약 임베딩은 문서의 핵심 내용을 신속하게 파악하는데 큰 도움 (효율성) 
    3) 가설 질문 활용 
        - 각 문서에 대해 적합한 가설 질문을 만들고, 해당 질문에 기반한 임베딩 생성 
        - 특정 주제나 내용에 대해 깊이 있는 탐색을 원할 때 유용 
    4) 수동 추가 
        - 사용자가 문서 검색 시 고려해야 할 특정 질문이나 쿼리를 직접 추가 
        - 검색 과정에서 좀 더 세밀한 제어를 할 수 있음 (맞춤형 검색) 

** 실습 자료 **

소프트웨어정책연구소(SPRi), [AI Brief 스페셜] AI 에이전트 동향

- 저자: 유재흥(AI정책연구실 책임연구원), 이해수(AI정책연구실 선임연구원)
- 발생: 2024.12.10
- 링크: https://spri.kr/posts/view/23798?code=AI-Brief&s_year=&data_page=1
- 파일명: `SPRi AI Brief_Special_AI Agent_241209_F`

(실습 준비)
- 작업 폴더 하위에 `data` 폴더 생성 
- 링크에서 다운로드 받은 파일을 `data` 폴더에 업로드 

## 문서 전처리

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

In [4]:
loader = PyMuPDFLoader("data/SPRi AI Brief_Special_AI Agent_241209_F.pdf")
docs = loader.load()

len(docs)

27

In [5]:
print(docs[5].page_content[:500])

 
SPRi AI Brief Special |  2024-12월호
5
2. AI 에이전트 주요 사례 
1) Microsoft
£ (개요) Microsoft는 클라우드 플랫폼 Azure를 중심으로 강력한 AI 에이전트를 개발 및 
제공하며, 특히 OpenAI와의 파트너십을 통해 ChatGPT와 기타 AI 모델을 Azure에 
통합하여 다양한 상용화 제품을 출시
∙‘Power Virtual Agents’를 포함한 Microsoft의 제품군은 기업용 서비스, 사무 자동화, 개발자 지원 도구 
등 다양한 분야에 걸쳐 AI 에이전트와 같은 기능을 지원
※ Fortune 500 기업 중 60%가 Microsoft 365 Copilot을 사용하여 비즈니스 혁신과 조직의 역량을 강화10)
<Microsoft 제품군>  
*출처: microsoft.com
£ (기능/기술) Microsoft의 제품군은 다양한 플랫폼과 통합되어, 기업들이 AI 에이전트를 
활용해 업무 협업, 고객 응대 자동화, 챗봇 


## 검색 
MultiVectorRetriever 사용 

In [6]:
import uuid
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [7]:
vectorstore = Chroma(
    collection_name="small_bigger_chunks",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

# 부모 문서의 저장소 
store = InMemoryStore()

# 검색기 정의 
id_key = "doc_id"
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# 문서 ID 생성
doc_ids = [str(uuid.uuid4()) for _ in docs]
doc_ids

['3bd4baa9-7680-4f48-a00a-bd509323beee',
 'e6bb3405-ecd0-45d8-9dfc-7a07ad027984',
 'edc9dce0-ad8b-4ed7-903b-8eedff6d5459',
 'e80516e7-36eb-4ea1-804b-a7f46f959165',
 '55307035-87fe-4402-a578-14be59f83296',
 '0d818e9c-9edf-4934-9119-68bc6c0fc614',
 'c30d2b8f-0797-45c9-8cdf-76e1c7f4e823',
 '0ae9c5ac-1b29-47c5-ab83-ca0f186de3b0',
 'b9a9c772-adfe-41df-9dbb-d6c673c7f1db',
 '5caa2eb0-babe-4a18-bcd3-63817f48ba40',
 'dc3a40d2-4190-4b63-922e-3081bc1b5dfd',
 '28cc6ba1-cbe9-403f-b63e-b47ca56914a2',
 '7e956512-5bb9-4bb7-a7ed-e8971e2adde9',
 '335dc250-058e-49a0-9a11-42a9d28d3034',
 '14c62af5-f78a-4225-beef-9200814c5f44',
 '60011002-2f13-47d3-b07f-ae341e8ed788',
 'b93b917f-ef0f-4184-ad3e-e6182040ea6f',
 '3be5a321-7297-470b-aafc-547875cd78b3',
 'b93becbf-dcf8-48e1-b361-4bd8258c62bb',
 'fd2b7899-d378-4310-8716-b64faf5e9d2f',
 '8b2858b9-6fa4-4e70-8a09-db0557d60717',
 '0742b5ff-fce6-4075-8b34-5a82c9788cb6',
 '9797c6d2-8b5b-4b79-887d-f84419a2f61b',
 '24c97888-ac2d-4cf7-9f95-41980f5ee39e',
 'dcf3542b-4ce1-

In [8]:
len(doc_ids)

27

In [9]:
# Parent 분할기
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# Child 분할기
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [10]:
# Parent 문서 분할 
parent_docs = []

for i, doc in enumerate(docs):
    # 현재 문서의 ID
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    parent_doc = parent_text_splitter.split_documents([doc])

    for _doc in parent_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

In [11]:
# 생성된 Parent 문서의 메타데이터 확인
parent_docs[0].metadata

{'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
 'file_path': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
 'page': 0,
 'total_pages': 27,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13947',
 'producer': 'Hancom PDF 1.3.0.547',
 'creationDate': "D:20241209094315+09'00'",
 'modDate': "D:20241209094315+09'00'",
 'trapped': '',
 'doc_id': '3bd4baa9-7680-4f48-a00a-bd509323beee'}

In [12]:
# Child 문서 분할 
child_docs = []

for i, doc in enumerate(docs):
    # 현재 문서의 ID
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

In [13]:
# 생성된 Child 문서의 메타데이터 확인
child_docs[0].metadata

{'source': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
 'file_path': 'data/SPRi AI Brief_Special_AI Agent_241209_F.pdf',
 'page': 0,
 'total_pages': 27,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13947',
 'producer': 'Hancom PDF 1.3.0.547',
 'creationDate': "D:20241209094315+09'00'",
 'modDate': "D:20241209094315+09'00'",
 'trapped': '',
 'doc_id': '3bd4baa9-7680-4f48-a00a-bd509323beee'}

In [14]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 88
분할된 child_docs의 개수: 442


In [15]:
# 벡터 저장소에 parent + child 문서 추가
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# docstore 에 원본 문서를 저장
retriever.docstore.mset(list(zip(doc_ids, docs)))

### 유사도 검색 

In [16]:
question = "구글의 Project Astra는 무엇인가요?"

In [17]:
# vectorstore의 유사도 검색
relevant_chunks = retriever.vectorstore.similarity_search(question)

print(f"검색된 문서의 개수: {len(relevant_chunks)}")

검색된 문서의 개수: 4


In [18]:
for chunk in relevant_chunks:
    print(chunk.page_content, end="\n\n")
    print(">" * 100, end="\n\n")

∙2024년 4월 개최된 ‘Google I/O 2024’에서 미래의 AI 어시스턴트 개발 프로젝트 ‘Project Astra’을 발표
 - 일상생활을 위한 AI 에이전트로 불리는 프로젝트 아스트라(Project Astra)는 구글 렌즈와 같이 실시간 
비디오와 오디오를 이해하고, 추론하고, 반응할 수 있음
<구글의 고객 참여 및 대화 관련 AI 제품>

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

∙Gemini 모델을 기반으로 한 ‘Project Astra’는 다중모드 정보를 처리하고, 사용자의 상황을 이해하며, 
대화에서 자연스럽게 응답할 수 있는 AI 어시스턴트의 미래를 탐구
물리적 그림 설명
조망 그림 인식
물체 순서 기억
<구글의 Project Astra 활용 예시>
*출처: deepmind.google15)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

‘Project Astra’를 통해 보편적인 AI 에이전트를 개발 중 
∙Google은 AI 기술을 활용하여 사용자 경험을 극대화하고 다양한 산업에 적용 가능한 AI 에이전트로 
Google Assistant, Dialogflow, Duplex 등을 제공하며, 이러한 AI 에이전트는 자연어 처리와 음성 인식

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

기술, 대화형 AI 기술을 활용하여 엔터프라이즈급 생성형 AI 애플리케이션을 만들 수 있도록 지원
∙2024년 4월 개최된 ‘Google I/O 2024’에서 미래의 AI 어시스턴트 개발 프로젝트 ‘Project Astra’을 발표
 - 일상생활을

In [19]:
# 부모 문서 검색 결과
relevant_docs = retriever.invoke(question)

print(f"검색된 문서의 개수: {len(relevant_docs)}", end="\n\n")
print("=" * 100, end="\n\n")
print(relevant_docs[0].page_content)

검색된 문서의 개수: 3


SPRi AI Brief Special |  2024-12월호
8
£ (최신 동정) Google의 ‘Vertex AI Agent Builder’ 등 최근의 제품군들은 고급 챗봇 및 
대화형 AI 기능을 강화
∙Google 클라우드는 2024년 4월 라스베이거스에서 열린 ‘Google Cloud Next '24’에서 고급 챗봇 및 
기타 대화형 AI 기능을 보다 쉽게 ​생성할 수 있도록 지원하는 ‘Vertex AI Agent Builder’를 발표16) 
 - ‘Vertex AI Agent Builder’는 머신러닝 기술이 제한적인 개발자도 Google의 파운데이션 모델, 검색 전문 
기술, 대화형 AI 기술을 활용하여 엔터프라이즈급 생성형 AI 애플리케이션을 만들 수 있도록 지원
∙2024년 4월 개최된 ‘Google I/O 2024’에서 미래의 AI 어시스턴트 개발 프로젝트 ‘Project Astra’을 발표
 - 일상생활을 위한 AI 에이전트로 불리는 프로젝트 아스트라(Project Astra)는 구글 렌즈와 같이 실시간 
비디오와 오디오를 이해하고, 추론하고, 반응할 수 있음
<구글의 고객 참여 및 대화 관련 AI 제품> 
*출처: google.com17)
∙ ‘Gemini at Work’ 행사(2024년 9월 개최)에서 새로운 Gemini 모델(Gemini 1.5 Flash 및  1.5 Pro)과 
함께 고객 참여 및 대화를 위해 설계된 AI 에이전트 제품군 출시18)
 - ‘Gemini at Work’에서 Google Cloud CEO(Thomas Kurian)는 기조연설에서 고객 상담 AI 에이전트가 
고객과 자연스럽게 대화하고 상호작용 하여 올바른 답변을 제공할 것이라고 언급
∙2024년 11월, Google은 웹을 검색하는 차기 AI 에이전트인 ‘Jarvis AI’를 개발 중인하고 있는 것으로 확인19)
 - ‘Jarvis AI’는 사용자가 Google Chrome에서 웹 브라우징 작업을 자동화하여, AI 에이전트가 

### MMR, Max Marginal Relevance 

In [20]:
from langchain.retrievers.multi_vector import SearchType

In [21]:
# 검색 유형 설정
retriever.search_type = SearchType.mmr

# 관련 문서 전체 검색
print(retriever.invoke(question)[0].page_content)

SPRi AI Brief Special |  2024-12월호
8
£ (최신 동정) Google의 ‘Vertex AI Agent Builder’ 등 최근의 제품군들은 고급 챗봇 및 
대화형 AI 기능을 강화
∙Google 클라우드는 2024년 4월 라스베이거스에서 열린 ‘Google Cloud Next '24’에서 고급 챗봇 및 
기타 대화형 AI 기능을 보다 쉽게 ​생성할 수 있도록 지원하는 ‘Vertex AI Agent Builder’를 발표16) 
 - ‘Vertex AI Agent Builder’는 머신러닝 기술이 제한적인 개발자도 Google의 파운데이션 모델, 검색 전문 
기술, 대화형 AI 기술을 활용하여 엔터프라이즈급 생성형 AI 애플리케이션을 만들 수 있도록 지원
∙2024년 4월 개최된 ‘Google I/O 2024’에서 미래의 AI 어시스턴트 개발 프로젝트 ‘Project Astra’을 발표
 - 일상생활을 위한 AI 에이전트로 불리는 프로젝트 아스트라(Project Astra)는 구글 렌즈와 같이 실시간 
비디오와 오디오를 이해하고, 추론하고, 반응할 수 있음
<구글의 고객 참여 및 대화 관련 AI 제품> 
*출처: google.com17)
∙ ‘Gemini at Work’ 행사(2024년 9월 개최)에서 새로운 Gemini 모델(Gemini 1.5 Flash 및  1.5 Pro)과 
함께 고객 참여 및 대화를 위해 설계된 AI 에이전트 제품군 출시18)
 - ‘Gemini at Work’에서 Google Cloud CEO(Thomas Kurian)는 기조연설에서 고객 상담 AI 에이전트가 
고객과 자연스럽게 대화하고 상호작용 하여 올바른 답변을 제공할 것이라고 언급
∙2024년 11월, Google은 웹을 검색하는 차기 AI 에이전트인 ‘Jarvis AI’를 개발 중인하고 있는 것으로 확인19)
 - ‘Jarvis AI’는 사용자가 Google Chrome에서 웹 브라우징 작업을 자동화하여, AI 에이전트가 쇼핑이나  
항공편 예약과 같

### similarity_score_threshold

In [22]:
score_threshold = 0.4

In [23]:
# 검색 유형 설정
retriever.search_type = SearchType.similarity_score_threshold
retriever.search_kwargs = {"score_threshold": score_threshold}

# 관련 문서 전체를 검색
print(retriever.invoke(question)[0].page_content)

SPRi AI Brief Special |  2024-12월호
8
£ (최신 동정) Google의 ‘Vertex AI Agent Builder’ 등 최근의 제품군들은 고급 챗봇 및 
대화형 AI 기능을 강화
∙Google 클라우드는 2024년 4월 라스베이거스에서 열린 ‘Google Cloud Next '24’에서 고급 챗봇 및 
기타 대화형 AI 기능을 보다 쉽게 ​생성할 수 있도록 지원하는 ‘Vertex AI Agent Builder’를 발표16) 
 - ‘Vertex AI Agent Builder’는 머신러닝 기술이 제한적인 개발자도 Google의 파운데이션 모델, 검색 전문 
기술, 대화형 AI 기술을 활용하여 엔터프라이즈급 생성형 AI 애플리케이션을 만들 수 있도록 지원
∙2024년 4월 개최된 ‘Google I/O 2024’에서 미래의 AI 어시스턴트 개발 프로젝트 ‘Project Astra’을 발표
 - 일상생활을 위한 AI 에이전트로 불리는 프로젝트 아스트라(Project Astra)는 구글 렌즈와 같이 실시간 
비디오와 오디오를 이해하고, 추론하고, 반응할 수 있음
<구글의 고객 참여 및 대화 관련 AI 제품> 
*출처: google.com17)
∙ ‘Gemini at Work’ 행사(2024년 9월 개최)에서 새로운 Gemini 모델(Gemini 1.5 Flash 및  1.5 Pro)과 
함께 고객 참여 및 대화를 위해 설계된 AI 에이전트 제품군 출시18)
 - ‘Gemini at Work’에서 Google Cloud CEO(Thomas Kurian)는 기조연설에서 고객 상담 AI 에이전트가 
고객과 자연스럽게 대화하고 상호작용 하여 올바른 답변을 제공할 것이라고 언급
∙2024년 11월, Google은 웹을 검색하는 차기 AI 에이전트인 ‘Jarvis AI’를 개발 중인하고 있는 것으로 확인19)
 - ‘Jarvis AI’는 사용자가 Google Chrome에서 웹 브라우징 작업을 자동화하여, AI 에이전트가 쇼핑이나  
항공편 예약과 같

### similarity

In [24]:
# 검색 유형 설정 (k=1)
retriever.search_type = SearchType.similarity
retriever.search_kwargs = {"k": 1}

# 관련 문서 전체를 검색
print(len(retriever.invoke(question)))

1


## 요약본 저장 

In [25]:
import uuid
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [26]:
# 로더 초기화 
loader = PyMuPDFLoader("data/SPRi AI Brief_Special_AI Agent_241209_F.pdf")

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

# PDF 파일 로드 및 텍스트 분할 실행
split_docs = loader.load_and_split(text_splitter)
print(f"분할된 문서의 개수: {len(split_docs)}")

분할된 문서의 개수: 71


In [27]:
summary_chain = (
    {"doc": lambda x: x.page_content}
    # 문서 요약을 위한 프롬프트 템플릿 생성
    | ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in summarizing documents in Korean."),
            (
                "user",
                "Summarize the following documents in 3 sentences in bullet points format.\n\n{doc}",
            ),
        ]
    )
    # LLM 통한 요약 생성
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | StrOutputParser()
)

In [28]:
# 문서 배치 처리 (최대 10개씩 처리)
summaries = summary_chain.batch(split_docs, {"max_concurrency": 10})

In [29]:
# 요약본 길이 확인 
print(len(summaries))

# 원본 문서의 내용을 출력
print(split_docs[33].page_content, end="\n\n")

# 요약본 출력
print("[요약]")
print(summaries[33])

71
SPRi AI Brief Special |  2024-12월호
13
 - 새로운 아바타는 Meta Horizon OS(VR 운영 체제)와 Facebook, Instagram, Messenger에서 사용
 - Meta의 Horizon 팀 부사장(Aigerim Shorman)은 업그레이드된 아바타를 통해 사용자에게 눈 크기, 코 
모양, 체형과 같은 것을 미세 조정할 수 있는 새로운 방법을 제공할 것이라고 언급
∙2025년 후반에는 사용자가 생성 AI를 활용하는 프롬프트를 입력하고 원하는 것을 설명하여 아바타를 
생성할 수 있을 전망 
5) AWS(Amazon Web Services)25)
£ (개요) AWS는 다양한 산업에서 AI 기술을 활용하여 고객 서비스, 보안, 데이터 분석 등을 
지원하며, Amazon Lex, Polly, Transcribe를 통해 음성 인식, 텍스트 변환, 대화형 
인터페이스 기능을 제공하고, Amazon Q를 통해 생성형 AI 기반 어시스턴트 기능을 제공26) 
∙AWS는 Amazon의 클라우드 서비스 플랫폼으로, 고객 서비스, 보안, 데이터 분석 등 여러 산업에 걸쳐 
다양한 AI 기술을 활용

[요약]
- Meta는 새로운 아바타를 Meta Horizon OS와 Facebook, Instagram, Messenger에서 사용할 수 있도록 하며, 사용자에게 아바타의 외형을 세밀하게 조정할 수 있는 기능을 제공할 예정이다.
- Horizon 팀 부사장 Aigerim Shorman은 2025년 후반부터 사용자가 생성 AI를 통해 아바타를 생성할 수 있는 기능을 도입할 것이라고 밝혔다.
- AWS는 다양한 산업에서 AI 기술을 활용하여 고객 서비스, 보안, 데이터 분석 등을 지원하며, Amazon Lex, Polly, Transcribe와 같은 서비스를 통해 음성 인식 및 대화형 인터페이스 기능을 제공한다.


In [30]:
# 요약 정보를 저장할 VectorDB 생성
summary_vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

# 부모 문서를 저장할 저장소
store = InMemoryStore()

# 검색기 정의
id_key = "doc_id"
retriever = MultiVectorRetriever(
    vectorstore=summary_vectorstore, 
    byte_store=store, 
    id_key=id_key, 
)

# 문서 ID 생성
doc_ids = [str(uuid.uuid4()) for _ in split_docs]
print(len(doc_ids))

71


In [31]:
# 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체 생성
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

# 요약본의 문서의 개수
print(len(summary_docs))

71


In [32]:
# 요약 문서를 VectorDB 저장 
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

In [33]:
# 유사도 검색 수행
result_docs = summary_vectorstore.similarity_search(question)

In [34]:
# 1개의 요약본 결과 문서
print(result_docs[0].page_content)

- Google은 'Vertex AI Agent Builder'를 통해 고급 챗봇과 대화형 AI 기능을 생성할 수 있는 도구를 제공하고 있다.
- 'Project Astra'는 Gemini 모델을 기반으로 하여 다중모드 정보를 처리하고 사용자의 상황을 이해하는 AI 어시스턴트를 개발하는 프로젝트이다.
- Google의 AI 에이전트는 일정 관리, 챗봇, 전화 예약 등 다양한 작업을 지원하며, Google Assistant와 Dialogflow와 같은 플랫폼을 통해 사용자와의 상호작용을 강화하고 있다.


In [35]:
# 요약본을 만든 관련된 문서 검색
retrieved_docs = retriever.invoke(question)
print(retrieved_docs[0].page_content)

SPRi AI Brief Special |  2024-12월호
7
∙최근 고급 챗봇과 대화형 AI 기능울 ​생성할 수 있도록 지원하는 도구로 ‘Vertex AI Agent Builder’ 제공
∙Gemini 모델을 기반으로 한 ‘Project Astra’는 다중모드 정보를 처리하고, 사용자의 상황을 이해하며, 
대화에서 자연스럽게 응답할 수 있는 AI 어시스턴트의 미래를 탐구
물리적 그림 설명
조망 그림 인식
물체 순서 기억
<구글의 Project Astra 활용 예시>
*출처: deepmind.google15)
£ (기능/기술) Google의 AI 에이전트들은 일정 관리, 챗봇, 전화 예약 등 다양한 작업을 지원
∙(Google Assistant) Google의 대표적인 AI 에이전트로, 사용자와의 음성 및 텍스트 대화를 통해 일상 
업무를 지원하고, 스마트폰, 스피커, 스마트 디스플레이 등 다양한 디바이스에서 사용 가능하며, 일정 관리, 
정보 검색, 스마트홈 기기 제어, 번역 등의 기능을 제공
∙(Dialogflow) 비즈니스용 AI 챗봇을 쉽게 개발할 수 있는 Google의 대화형 AI 플랫폼으료, Dialogflow를


## Hypothetical Query 

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_openai import ChatOpenAI

In [37]:
functions = [
    {
        "name": "hypothetical_questions",  
        "description": "Generate hypothetical questions", 
        "parameters": { 
            "type": "object",  
            "properties": {  
                "questions": {  
                    "type": "array",  
                    "items": {
                        "type": "string"
                    }, 
                },
            },
            "required": ["questions"],  
        },
    }
]

In [38]:
hypothetical_query_chain = (
    {"doc": lambda x: x.page_content}
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer. "
        "Potential users are those interested in the AI industry. Create questions that they would be interested in. "
        "Output should be written in Korean:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").bind(
        functions=functions, 
        function_call={"name": "hypothetical_questions"}
    )
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [39]:
# 주어진 문서에 대해 체인을 실행
hypothetical_query_chain.invoke(split_docs[33])

['Meta의 새로운 아바타 기능이 사용자 경험에 미치는 영향은 무엇일까?',
 'AWS의 AI 기술이 향후 기업의 고객 서비스 방식에 어떤 변화를 가져올까?',
 '2025년 후반에 사용자가 생성 AI를 활용하여 아바타를 생성하는 과정에서 발생할 수 있는 윤리적 문제는 무엇일까?']

In [40]:
# 문서 목록에 대해 가설 질문을 배치 생성
hypothetical_questions = hypothetical_query_chain.batch(
    split_docs, {"max_concurrency": 10}
)

In [41]:
hypothetical_questions[33]

['새로운 아바타 기술이 사용자 경험에 어떤 영향을 미칠까요?',
 'AWS의 AI 기술이 다양한 산업에서 어떻게 활용될 수 있을까요?',
 '사용자가 생성 AI를 통해 자신만의 아바타를 만드는 과정에서 어떤 창의적인 가능성이 열릴까요?']

In [42]:
# Child 청크를 인덱싱하는 데 사용할 벡터 저장소
hypothetical_vectorstore = Chroma(
    collection_name="hypo-questions", 
    embedding_function=OpenAIEmbeddings()
)

# Parent 문서의 저장소 계층
store = InMemoryStore()

# 검색기 정의
id_key = "doc_id"
retriever = MultiVectorRetriever(
    vectorstore=hypothetical_vectorstore,
    byte_store=store,
    id_key=id_key,
)

# 문서 ID 생성
doc_ids = [str(uuid.uuid4()) for _ in split_docs]  

In [43]:
question_docs = []
# hypothetical_questions 저장
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        # 질문 리스트의 각 질문에 대해 Document 객체를 생성하고, 메타데이터에 해당 질문의 문서 ID 포함
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

In [44]:
# hypothetical_questions 문서를 벡터 저장소에 추가
retriever.vectorstore.add_documents(question_docs)

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

In [45]:
# 유사한 문서 검색
result_docs = hypothetical_vectorstore.similarity_search(question)

# 유사도 검색 결과
for doc in result_docs:
    print(doc.page_content)
    print(doc.metadata)

'Project Astra'와 같은 미래의 AI 어시스턴트가 우리의 일상생활에 어떤 변화를 가져올 수 있을까요?
{'doc_id': 'f1083486-789d-4d4c-81a9-c6ce59b79f70'}
Project Astra의 다중모드 정보 처리 기능이 향후 AI 어시스턴트의 사용자 경험에 어떤 영향을 미칠까?
{'doc_id': '33410822-11ea-4bf2-95e4-76fca72abde1'}
구글의 AI 에이전트가 고객 참여에 어떻게 혁신을 가져올 수 있을까요?
{'doc_id': '5df4ae08-78a0-499c-af0c-5156c54ceee2'}
구글 클라우드의 새로운 Gemini 모델이 고객 참여 방식에 어떤 혁신을 가져올까요?
{'doc_id': '1849882b-7a3b-4cac-99b2-69dbe52cbddf'}


In [46]:
# 관련된 문서 검색
retrieved_docs = retriever.invoke(result_docs[1].page_content)

# 검색된 문서
for doc in retrieved_docs:
    print(doc.page_content)

SPRi AI Brief Special |  2024-12월호
7
∙최근 고급 챗봇과 대화형 AI 기능울 ​생성할 수 있도록 지원하는 도구로 ‘Vertex AI Agent Builder’ 제공
∙Gemini 모델을 기반으로 한 ‘Project Astra’는 다중모드 정보를 처리하고, 사용자의 상황을 이해하며, 
대화에서 자연스럽게 응답할 수 있는 AI 어시스턴트의 미래를 탐구
물리적 그림 설명
조망 그림 인식
물체 순서 기억
<구글의 Project Astra 활용 예시>
*출처: deepmind.google15)
£ (기능/기술) Google의 AI 에이전트들은 일정 관리, 챗봇, 전화 예약 등 다양한 작업을 지원
∙(Google Assistant) Google의 대표적인 AI 에이전트로, 사용자와의 음성 및 텍스트 대화를 통해 일상 
업무를 지원하고, 스마트폰, 스피커, 스마트 디스플레이 등 다양한 디바이스에서 사용 가능하며, 일정 관리, 
정보 검색, 스마트홈 기기 제어, 번역 등의 기능을 제공
∙(Dialogflow) 비즈니스용 AI 챗봇을 쉽게 개발할 수 있는 Google의 대화형 AI 플랫폼으료, Dialogflow를
SPRi AI Brief Special |  2024-12월호
8
£ (최신 동정) Google의 ‘Vertex AI Agent Builder’ 등 최근의 제품군들은 고급 챗봇 및 
대화형 AI 기능을 강화
∙Google 클라우드는 2024년 4월 라스베이거스에서 열린 ‘Google Cloud Next '24’에서 고급 챗봇 및 
기타 대화형 AI 기능을 보다 쉽게 ​생성할 수 있도록 지원하는 ‘Vertex AI Agent Builder’를 발표16) 
 - ‘Vertex AI Agent Builder’는 머신러닝 기술이 제한적인 개발자도 Google의 파운데이션 모델, 검색 전문 
기술, 대화형 AI 기술을 활용하여 엔터프라이즈급 생성형 AI 애플리케이션을 만들 수 있도록 지원
∙2024년 4월 개최된 ‘Google I/O 2024’에

-----
** End of Documents **